In [1]:
from modules.data_loading import load_train_data, load_test_data
import pandas as pd
import numpy as np
import ydata_profiling

In [2]:
df = load_train_data()
df.head()

,Unnamed: 0,ID,Prix,Un. rés.,Rev. brut. pot.,YearBuilt,Type de bâtiment,Dimensions du bâtiment,Éval. terrain,Superficie habitable,...,Foyers-Poêles,Équip./Serv.,Rénovations,Inclusions,Exclusions,Remarques - Courtier,Addenda,Nbre pièces,Nbre chambres (hors-sol + sous-sol),Nbre salles de bains + salles d'eau
0,0,1,1 000 000 $,3,46 200 $,Triplex dans le quartier Saint-Henri/Petite Bo...,En rangée,NaN,254 400 $,"2 972,99 pc / 276,2 mc",...,NaN,"Ascenseur(s), Buanderie",NaN,NaN,Effets personnels des locataires,Ne manquez pas cette incroyable opportunité de...,"Bienvenue à St-Henri, un quartier dynamique co...",8.0,3.0,1+0
1,1,2,725 000 $,3,37 200 $,Triplex dans le quartier Saint-Henri/Petite Bo...,En rangée,"25 X p irr / 7,62 X m irr",254 400 $,"2 895 pc / 268,95 mc",...,NaN,NaN,NaN,NaN,NaN,Triplex situé au coeur du très convoité secteu...,CARACTÉRISTIQUES :+Stationnement extérieur+Rev...,5.0,3.0,1+0
2,2,3,725 000 $,3,37 500 $,Triplex dans le quartier Saint-Henri/Petite Bo...,En rangée,"25 X p irr / 7,62 X m irr",254 400 $,"2 897,64 pc / 269,2 mc",...,NaN,NaN,NaN,NaN,NaN,Triplex situé au coeur du très convoité secteu...,"LOCALISATION+ Emplacement convoité, près de to...",5.0,3.0,1+0
3,3,4,942 000 $,3,54 660 $,Triplex dans le quartier Saint-Henri/Petite Bo...,En rangée,"25 X 58,2 p irr / 7,62 X 17,73 m irr",254 400 $,NaN,...,NaN,Thermopompe murale,NaN,"Cuisinière, réfrigérateur, lave-vaisselle, lav...","Cuisinière, réfrigérateur, lave-vaisselle, lav...","Triplex dans Saint-Henri, un secteur très rech...","Les visites débuteront le jeudi 17 novembre, 2...",5.0,3.0,1+0
4,4,5,732 000 $,2,32 400 $,Duplex dans le quartier Saint-Paul/Émard const...,En rangée,"25 X 45 p / 7,62 X 13,72 m",149 400 $,"2 292 pc / 212,93 mc",...,NaN,NaN,NaN,"5933 : Fixtures, stores, rideaux et pôles, hot...",5933 : Tous les biens du locataires,Opportunité intéressante pour investisseurs ou...,"Voici le 5933-5935, un duplex situé sur une ru...",5.0,3.0,1+0


In [3]:
null_cols = df.columns[df.isnull().any()]
null_cols

Index(['Dimensions du bâtiment', 'Éval. terrain', 'Superficie habitable',
       'Éval. bâtiment', 'Dimensions du terrain', 'Superficie du terrain',
       'Plan d'eau', 'Piscine', 'Stationnement (total)', 'Chauffage',
       'Eau (accès)', 'Foyers-Poêles', 'Équip./Serv.', 'Rénovations',
       'Inclusions', 'Exclusions', 'Remarques - Courtier', 'Addenda',
       'Nbre pièces', 'Nbre chambres (hors-sol + sous-sol)',
       'Nbre salles de bains + salles d'eau', 'Source'],
      dtype='object')

In [4]:
profile = df.profile_report()
profile.to_file("eda_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]